# JSON Mini Project

In [134]:
# import modules
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize

## 1. Find the countries with most projects

In [135]:
# open the json file and read it as a DataFrame
with open('data/world_bank_projects.json','r') as json_file:
    json_df = pd.read_json(json_file)

In [136]:
# what do the data look like?
json_df.head(1)

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...


In [137]:
# which columns will be useful to answer the question?
json_df.columns

Index(['_id', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract', 'project_name', 'projectdocs',
       'projectfinancialtype', 'projectstatusdisplay', 'regionname', 'sector',
       'sector1', 'sector2', 'sector3', 'sector4', 'sector_namecode',
       'sectorcode', 'source', 'status', 'supplementprojectflg', 'theme1',
       'theme_namecode', 'themecode', 'totalamt', 'totalcommamt', 'url'],
      dtype='object')

In [138]:
# group the projects by countries and sort the number of projects in descending order
countries_S = json_df.groupby('countryname').size().sort_values(ascending=False)

# print the first 10 countries
countries_S.head(10)

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
dtype: int64

## 2. Find the top 10 major project themes (using column 'mjtheme_namecode')

In [139]:
# what does this column looks like?

df = json_df['mjtheme_namecode']
df.head()

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object

In [140]:
# try normalize with the first row

json_normalize(df[0])

,code,name
0,8,Human development
1,11,


In [141]:
# create an empty theme dataframe and append each normalized dataframe
# keep project _id from original json (may be useful later)

theme_df = pd.DataFrame()
for i in range(len(df)):
    _id = str(json_df.iloc[i,0])
    theme = json_normalize(df[i])
    theme['_id'] = _id
    theme_df = theme_df.append(theme)

In [142]:
theme_df.head()

,code,name,_id
0,8,Human development,{'$oid': '52b213b38594d8a2be17c780'}
1,11,,{'$oid': '52b213b38594d8a2be17c780'}
0,1,Economic management,{'$oid': '52b213b38594d8a2be17c781'}
1,6,Social protection and risk management,{'$oid': '52b213b38594d8a2be17c781'}
0,5,Trade and integration,{'$oid': '52b213b38594d8a2be17c782'}


In [143]:
# Group the theme dataframe by code, count the number of lines, sort descending and print the 10 first.

theme_df.groupby('code').size().sort_values(ascending=False).head(10)

code
11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
dtype: int64

## 3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [144]:
# Replace empty strings by NaN

theme_df = theme_df.replace('', np.nan)
theme_df.head()

,code,name,_id
0,8,Human development,{'$oid': '52b213b38594d8a2be17c780'}
1,11,NaN,{'$oid': '52b213b38594d8a2be17c780'}
0,1,Economic management,{'$oid': '52b213b38594d8a2be17c781'}
1,6,Social protection and risk management,{'$oid': '52b213b38594d8a2be17c781'}
0,5,Trade and integration,{'$oid': '52b213b38594d8a2be17c782'}


In [145]:
# Check how many NaN there are

theme_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1499 entries, 0 to 3
Data columns (total 3 columns):
code    1499 non-null object
name    1377 non-null object
_id     1499 non-null object
dtypes: object(3)
memory usage: 46.8+ KB


In [146]:
# Sort the dataframe

df_sort = theme_df.sort_values(['code','name'])

# print df for code == '1' to see what it looks like

df_sort[df_sort['code'] == '1'].tail(10)

,code,name,_id
0,1,Economic management,{'$oid': '52b213b38594d8a2be17c919'}
1,1,Economic management,{'$oid': '52b213b38594d8a2be17c919'}
1,1,Economic management,{'$oid': '52b213b38594d8a2be17c91a'}
0,1,Economic management,{'$oid': '52b213b38594d8a2be17c923'}
1,1,Economic management,{'$oid': '52b213b38594d8a2be17c924'}
1,1,NaN,{'$oid': '52b213b38594d8a2be17c7ce'}
1,1,NaN,{'$oid': '52b213b38594d8a2be17c802'}
1,1,NaN,{'$oid': '52b213b38594d8a2be17c8da'}
1,1,NaN,{'$oid': '52b213b38594d8a2be17c8f7'}
1,1,NaN,{'$oid': '52b213b38594d8a2be17c95f'}


In [147]:
# Since the NaN are sorted at the end, I will use ffill

df_sort = df_sort.fillna(method='ffill')

# Print the result for code == '1'

df_sort[df_sort['code'] == '1'].tail(10)

,code,name,_id
0,1,Economic management,{'$oid': '52b213b38594d8a2be17c919'}
1,1,Economic management,{'$oid': '52b213b38594d8a2be17c919'}
1,1,Economic management,{'$oid': '52b213b38594d8a2be17c91a'}
0,1,Economic management,{'$oid': '52b213b38594d8a2be17c923'}
1,1,Economic management,{'$oid': '52b213b38594d8a2be17c924'}
1,1,Economic management,{'$oid': '52b213b38594d8a2be17c7ce'}
1,1,Economic management,{'$oid': '52b213b38594d8a2be17c802'}
1,1,Economic management,{'$oid': '52b213b38594d8a2be17c8da'}
1,1,Economic management,{'$oid': '52b213b38594d8a2be17c8f7'}
1,1,Economic management,{'$oid': '52b213b38594d8a2be17c95f'}


In [148]:
# Check if there is no mistake

df_check = df_sort[['code','name']].drop_duplicates()
df_check

,code,name
0,1,Economic management
0,10,Rural development
2,11,Environment and natural resources management
1,2,Public sector governance
1,3,Rule of law
1,4,Financial and private sector development
0,5,Trade and integration
1,6,Social protection and risk management
0,7,Social dev/gender/inclusion
0,8,Human development


In [149]:
df_sort.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1499 entries, 0 to 1
Data columns (total 3 columns):
code    1499 non-null object
name    1499 non-null object
_id     1499 non-null object
dtypes: object(3)
memory usage: 46.8+ KB


In [150]:
# Answer question 2 with theme names

df_sort.groupby('name').size().sort_values(ascending=False).head(10)

name
Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
dtype: int64